# Identificacion de mas de una tabla en una misma hoja excel

### Set API MODEL GEMINI 1.5 PRO

In [1]:
import vertexai
import os
from dotenv import load_dotenv
from langchain_google_vertexai import ChatVertexAI

vertexai.init(
    project="gen-lang-client-0730997933",
    location="us-central1"
)

os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = 'gen-lang-client-0730997933-9e99e8511560.json'

load_dotenv()

GOOGLE_API_KEY = os.getenv('GOOGLE_API_KEY')

os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY

llm_gemini_pro = ChatVertexAI(model="gemini-1.5-pro-002")

### Prompt

In [2]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

# ID para guardar el historial de consultas anteriores
MEMORY_KEY = "chat_history"

extract_dataexcel_name_tool = 'get_tables'

# Template configurado para el prompt
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            f"""System prompt:
                    Tu objetivo es mediante una herramienta entregada para extraer los datos de un excel, obtener las tablas que se encuentran en este y ademas relacionarlas si es que tienen alogun tipo de atributo en comun.

                    Para ello tienes que seguir estos pasos:
                        1. Extraer los datos de un excel con la herramienta llamada {extract_dataexcel_name_tool}.
                        2. Guarda los datos en un diccionario de datos.
                        3. Interpreta los datos e identifica cuales son los datos en comun entre tablas.
                        4. Genera un script sql para crear las tablas en la base de datos.

                    Como salida debes entregar el script y un esquema con las tablas y las conexiones entre ellas.
            """,
        ),
        MessagesPlaceholder(variable_name=MEMORY_KEY),
        ("user", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)

### Tools

In [3]:
from langchain_core.tools import tool
from typing import List, Type, Dict
import pandas as pd

@tool
def get_tables(file_path: str) -> Dict[str, List[Dict[str, str]]]:
    """Extrae tablas desde un archivo Excel, guarda cada tabla en un CSV y devuelve un diccionario con columnas y tipos de datos."""
    xls = pd.ExcelFile(file_path)
    tables_info = {}

    for sheet_name in xls.sheet_names:
        df = xls.parse(sheet_name)
        tables_in_sheet = []
        
        # Dividir en tablas usando filas vacías como separación
        start_idx = 0
        for i, is_empty in enumerate(df.isnull().all(axis=1)):
            if is_empty:
                # Si encontramos una fila vacía, extraemos la tabla hasta el índice actual
                if start_idx < i:
                    sub_df = df.iloc[start_idx:i].dropna(how="all")
                    tables_in_sheet.append(sub_df)
                start_idx = i + 1  # Actualizar inicio para la siguiente tabla
                
        # Última tabla (si hay contenido después de la última fila vacía)
        if start_idx < len(df):
            tables_in_sheet.append(df.iloc[start_idx:].dropna(how="all"))

        # Guardar cada tabla en CSV y obtener info de columnas
        tables_info[sheet_name] = []
        for idx, table_df in enumerate(tables_in_sheet):
            csv_file_name = f"{sheet_name}_table_{idx + 1}.csv"
            table_df.to_csv(csv_file_name, index=False)
            
            columns_info = [
                {"name": column, "dtype": str(dtype)}
                for column, dtype in table_df.dtypes.items()
            ]
            tables_info[sheet_name].append(columns_info)
    
    return tables_info

tools = [get_tables]

llm_with_tools = llm_gemini_pro.bind_tools(tools)

### Agent

In [4]:
from langchain.agents.format_scratchpad.openai_tools import (
    format_to_openai_tool_messages,
)
from langchain.agents.output_parsers.openai_tools import OpenAIToolsAgentOutputParser

agent = (
    {
        "input": lambda x: x["input"],
        "agent_scratchpad": lambda x: format_to_openai_tool_messages(
            x["intermediate_steps"]
        ),
        "chat_history": lambda x: x["chat_history"],
    }
    | prompt
    | llm_with_tools
    | OpenAIToolsAgentOutputParser()
)

### Agent Executor

In [5]:
from langchain.agents import AgentExecutor

chat_history = []

agent_executor = AgentExecutor(agent=agent, tools = tools, verbose=True)

### Invoke agent

In [6]:
input = "Genera un script sql desde el archivo ubicado en './Book1.xlsx'"
result = agent_executor.invoke({"input": input,"chat_history": chat_history})



> Entering new AgentExecutor chain...

Invoking: `get_tables` with `{'file_path': './Book1.xlsx'}`


{'Campañas_promociones': [[{'name': 'Código de campaña', 'dtype': 'object'}, {'name': 'Nombre campaña', 'dtype': 'object'}, {'name': 'Objetivo', 'dtype': 'object'}, {'name': 'Monto descuento', 'dtype': 'float64'}, {'name': 'Tipo de descuento', 'dtype': 'object'}, {'name': 'Categoría producto', 'dtype': 'object'}, {'name': 'Fecha inicio', 'dtype': 'datetime64[ns]'}, {'name': 'Fecha de término', 'dtype': 'datetime64[ns]'}]], 'campañas_2': [[{'name': 'Código de campaña', 'dtype': 'object'}, {'name': 'Nombre campaña', 'dtype': 'object'}, {'name': 'Objetivo', 'dtype': 'object'}, {'name': 'Monto descuento', 'dtype': 'object'}, {'name': 'Tipo de descuento', 'dtype': 'object'}, {'name': 'Categoría producto', 'dtype': 'object'}, {'name': 'Fecha inicio', 'dtype': 'datetime64[ns]'}, {'name': 'Fecha de término', 'dtype': 'datetime64[ns]'}, {'name': 'Ventas en el periodo', 'dtype': 'int64'}, {'nam

In [7]:
print(result.get('output'))

```sql
CREATE TABLE Campañas_promociones (
    "Código de campaña" VARCHAR(255),
    "Nombre campaña" VARCHAR(255),
    Objetivo VARCHAR(255),
    "Monto descuento" FLOAT,
    "Tipo de descuento" VARCHAR(255),
    "Categoría producto" VARCHAR(255),
    "Fecha inicio" DATE,
    "Fecha de término" DATE
);

CREATE TABLE campañas_2 (
    "Código de campaña" VARCHAR(255),
    "Nombre campaña" VARCHAR(255),
    Objetivo VARCHAR(255),
    "Monto descuento" VARCHAR(255),
    "Tipo de descuento" VARCHAR(255),
    "Categoría producto" VARCHAR(255),
    "Fecha inicio" DATE,
    "Fecha de término" DATE,
    "Ventas en el periodo" INT,
    "Unnamed: 9" FLOAT,
    "Unnamed: 10" FLOAT,
    "Unnamed: 11" VARCHAR(255),
    "Unnamed: 12" VARCHAR(255)
);
```

Las tablas `Campañas_promociones` y `campañas_2` comparten las siguientes columnas: `"Código de campaña"`, `"Nombre campaña"`, `"Objetivo"`, `"Monto descuento"`, `"Tipo de descuento"`, `"Categoría producto"`, `"Fecha inicio"` y `"Fecha de término"`.

In [8]:
get_tables('./Book1.xlsx')

C:\Users\jock_\AppData\Local\Temp\ipykernel_19752\1584369368.py:1: LangChainDeprecationWarning: The method `BaseTool.__call__` was deprecated in langchain-core 0.1.47 and will be removed in 1.0. Use :meth:`~invoke` instead.
  get_tables('./Book1.xlsx')


{'Campañas_promociones': [[{'name': 'Código de campaña', 'dtype': 'object'},
   {'name': 'Nombre campaña', 'dtype': 'object'},
   {'name': 'Objetivo', 'dtype': 'object'},
   {'name': 'Monto descuento', 'dtype': 'float64'},
   {'name': 'Tipo de descuento', 'dtype': 'object'},
   {'name': 'Categoría producto', 'dtype': 'object'},
   {'name': 'Fecha inicio', 'dtype': 'datetime64[ns]'},
   {'name': 'Fecha de término', 'dtype': 'datetime64[ns]'}]],
 'campañas_2': [[{'name': 'Código de campaña', 'dtype': 'object'},
   {'name': 'Nombre campaña', 'dtype': 'object'},
   {'name': 'Objetivo', 'dtype': 'object'},
   {'name': 'Monto descuento', 'dtype': 'object'},
   {'name': 'Tipo de descuento', 'dtype': 'object'},
   {'name': 'Categoría producto', 'dtype': 'object'},
   {'name': 'Fecha inicio', 'dtype': 'datetime64[ns]'},
   {'name': 'Fecha de término', 'dtype': 'datetime64[ns]'},
   {'name': 'Ventas en el periodo', 'dtype': 'int64'},
   {'name': 'Unnamed: 9', 'dtype': 'float64'},
   {'name': 'Un